# [E-04] To make lyricist 

## Contexts

### 1. READY
    1-1 오늘의 Exp와 Rubric  
    1-2 사용하는 라이브러리  

### 2. GAME
    2-1. 데이터 읽어오기  
    2-2. 데이터 전처리
        2-2-1. Tokenize (문장의 정형화)
        2-2-2. Tensor (리스트 객체를 텐서 객체로 변환)
        2-2-3. 데이터 분리 (Train, Validation)
        2-2-4. 데이터셋 화 (텐서 객체를 dataset 객체로 변환)
    2-3. 모델 학습  
    2-4. 데이터 평가   

### 3. POTG
    3-1. 소감(POTG)  
    3-2. 어려웠던 점과 극복방안  
    3-3. 추후  

---




## 1. READY

   ### 1-1. 오늘의 EXP 와 Rubric

오늘의 EXP 내용은 NLP (Natural Language Processing) 기술을 이용한 인공지능 작사 모델 생성이다.

- 데이터 : 미국 유명 아티스트 50명들의 곡 가사정보를 담은 텍스트 파일
<img src="./img/lyric.PNG" width="200px"></img>


- 모델 : tensor - Adam

- rubric 제시

|평가문항|상세기준|
|---|---|
|1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?|텍스트 제너레이션 결과가 그럴듯한 문장으로 생서되는지|
|2. 데이터의 전처리/ 데이터셋 구성 과정이 체계적으로 진행되는가?| 특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되는지|
|3. 텍스트 생성모델이 잘 학습되었는지|텍스트 생성모델의 validation loss가 2.2 이하인지|

### 1-2. 사용하는 라이브러리

In [32]:
import glob
import os
import re
import tensorflow as tf #tf.keras.preprocessing.text.Tokenizer

from sklearn.model_selection import train_test_split
import numpy as np
import pandas

- glob : 파일 핸들링 라이브러리
- os : 경로 설정 라이브러리
- re : 정규표현식 라이브러리
- tf : 인공지능 모델 생성, 학습, 저장 라이브러리  
  tf.keras.preprocessing.text.Tokenizer :(토큰화 / 단어사전 생성 / 숫자변환) 벡터화
- train_test_split : 모은 데이터를 분리하는 라이브러리
- np / pandas : 데이터 구성을 이루는 라이브러리

---

# 2. GAME
## 2-1. 데이터 읽어오기

In [1]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel//Study/Exp_4_lyric/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


자 리스트 안에 문장별로 쭉 str데이터가 들어갔다.

## 2-2. 데이터 전처리
### 2-2-1. Tokenize

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1 소문자화
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2 특수문자 양쪽에 공백
    sentence = re.sub(r'[" "]+', " ", sentence) # 3 여러 공백을 하나의 공백으로
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4 알파벳, ?.!,¿가 아닌 모든 문자를 하나의 공백으로
    sentence = sentence.strip() # 5 양쪽 공백 지우기
    sentence = '<start> ' + sentence + ' <end>' # 6 
    return sentence

raw_corpus[0].split(' ')

['Now', "I've", 'heard', 'there', 'was', 'a', 'secret', 'chord']

preprocess_sentence() 함수로 데이터를 정형화 시켜야 한다.

* preprocess_sentence() 는  
  - 문장부호를 띄워 놓기
  - 전부 소문자화
  - 특수문자 제거
  - start , end 추가  
  
를 통해 모든 문장을 동일한 규칙성을 띠게 한다.

In [3]:

# 여기에 정제된 문장을 모을겁니다
corpus = []


excepted = 0  # 제외되는 양
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    
    if len(preprocessed_sentence.split()) > 15:
        excepted += 1
        continue
    corpus.append(preprocessed_sentence)

print(excepted, '개는 너무 길어서 제외')

len(corpus)

19736 개는 너무 길어서 제외


156013


만든 함수를 이용하기 전
- 아무것도 없는 줄
- : 로 끝나는 문장(가수나 상대가수의 이름이 있다고 추측)  

을 제거한 다음 메서드로 정형화를 시켰다, 그리고 정제된 문장 중  
단어가 15개를 넘는 문장들(start, end 포함) 을 제하고 문장리스트를 만들었다.

### 2-2-2. Tensor 

우리의 데이터를 학습시키기 위해서는 우리의 tokenizer 데이터를  
tensor 객체로 변환시켜주어야 한다.


In [4]:
import tensorflow as tf

def tokenize(corpus):
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f22905e3f40>


In [5]:
tensor[0]

array([   2,   50,    4,   95,  303,   62,   53,    9,  946, 6263,    3,
          0,    0,    0,    0], dtype=int32)

그 객체와 함께 쓰이는 문장사전(12000개로 구성)을 Tokenizer 메서드를 통해 불렀고,  
우리가 모은 문장 리스트를 통해 필요한 문장을 사전에 구축하였다.

추가로
- tokenizer.texts_to_sequences 메서드를 통해 우리의 문장 리스트를 텐서화 했다.  
- tf.keras.preprocessing.sequence.pad_sequences메서드를 통해 문장별 token 수를 맞춘다.

텐서의 src , target 데이터를 다음과 같다.

src 인스턴스는 엔딩이 없는 시작 데이터 (문제지) 이고  
tgt 인스턴스는 스타트가 없는 마무리 데이터 (답안지) 이다.

이 두 데이터를 연결지으며 다음에 올 문장을 예측한다.

**이것이 RNN 의 기본 메커니즘이다.**

In [6]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


### 2-2-3. 데이터셋을 train, validation 값으로 분리

NLP 는 목적성(새로운 작사를 하려는 의도) 에 따라 test 값으로 측정하기가 애매하다.  
그러므로 우리에게 필요한 데이터 구성은

- 데이터를 학습시키는 Train 데이터
- 데이터를 학습시키며 하이퍼 파라미터를 조정해주는 Validation 데이터

In [7]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=42)

sklearn 모듈의 train_test_split 메서드로 0.2의 비율로 분리했다.

In [8]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


위와 같이 14개의 token을 가진 124810개의 문장으로 구성되어 있다.

### 2-2-4. dataset

모델 학습 직전 그 모델에 맞도록 데이터에 일정 파라미터 값을 설정하고 변환시켜야 한다.

1. buffer size : 넣을 데이터의 양 (124810)
2. batch size : 수행할 미니배치의 크기 (256개씩 묶을 것이다)
3. steps per epoch : epoch마다 학습할 횟수 = (버퍼사이즈를 미니배치로 나눈 만큼)

* tf.data.Dataset.from_tensor_slices 메서드를 통해  
tensor에서 dataset 으로 데이터를 변환한다.


* shuffle 메서드와 batch 메서드로 설정한 파라미터값을 넣어준다.

In [9]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE
print(steps_per_epoch)

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함
VOCAB_SIZE = tokenizer.num_words + 1   

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)) #이걸로 tensor의 train 데이터를 tf.data.dataset 객체로 변환하여 사용할거임
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
print(dataset)


dataset_val = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)) #이걸로 tensor의 validation 데이터를 tf.data.dataset 객체로 변환하여 사용할거임
dataset_val = dataset_val.shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)
print(dataset_val)

487
<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


---


## 2-3. 모델 학습



In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 2048

model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

TextGenerator 클래스는 우리가 만들 모델의 작동원리를 나타낸다 .  
내부에는 임베딩 레이어, 2개의 RNN 레이어, 1개의 Dense 레이어가 있다.  

- embedding size : 워드 벡터의 차원 수  
- hidden size : LSTM레이어의 hidden state 차원 수

In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

tf.test.is_gpu_available()


model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10, validation_data=dataset_val)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Epoch 1/10
487/487 [==============================] - 289s 518ms/step - loss: 3.2777 - val_loss: 2.9099
Epoch 2/10
487/487 [==============================] - 262s 537ms/step - loss: 2.7318 - val_loss: 2.6511
Epoch 3/10
487/487 [==============================] - 262s 538ms/step - loss: 2.4125 - val_loss: 2.4685
Epoch 4/10
487/487 [==============================] - 262s 537ms/step - loss: 2.1014 - val_loss: 2.3304
Epoch 5/10
487/487 [==============================] - 261s 536ms/step - loss: 1.8100 - val_loss: 2.2380
Epoch 6/10
487/487 [==============================] - 261s 536ms/step - loss: 1.5535 - val_loss: 2.1693
Epoch 7/10
487/487 [==============================] - 262s 537ms/step - loss: 1.3446 - val_loss: 2.1385
Epoch 8/10
487/487 [==============================] - 261s 536ms/step - loss: 1.1872 - val_loss: 2.1399
Epoch 9/10
487/487 [==============================] - 261s 536ms/step - loss: 1.0818 - 

학습이 완료되었습니다  
최종 결과로는 loss : 1.34, val-loss : 2.14 를 채택할 수 있다! 

### 2-4. 데이터 평가
말한 바와 같이 NLP 의 경우에는 테스트 데이터가 따로 없기에  
만들어진 모델로 직접 글을 작성하여 마음에 드는지를 확인한다.

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

모델에서는 tensor 데이터를 사용하기 때문에  
변수를 입력하고 데이터를 빼올 때 항상 데이터객체 전환과정이 필요하다. 

위는 그 기능과 함께 최대 20단어까지 나오도록 설정한 함수이다.

In [18]:
generate_text(model, tokenizer, init_sentence="<start> I love")

'<start> i love you so much , so o o <end> '

너무 로맨틱하다, 작사에 맞게 작곡하고 싶은 욕구가 샘솟는다.

In [20]:
generate_text(model, tokenizer, init_sentence="<start> he")

'<start> he s a monster <end> '

어떤 의미의 괴물일지에 대한 궁금증이 등을 타고 흐른다.

In [21]:
generate_text(model, tokenizer, init_sentence="<start> she")

'<start> she s got me runnin round and round <end> '

나도 내 곁을 맴도는 그녀가 있었으면 좋겠다.

In [24]:
generate_text(model, tokenizer, init_sentence="<start> ha")

'<start> ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha ha '

In [29]:
generate_text(model, tokenizer, init_sentence="<start> finished work ,")

'<start> finished work , i m gonna watch this motherfuckin thing <end> '

난 다 했고, 이제 빌어먹을 넷플릭스 보러간다!


---



## 3. POTG

### 3-1. 소감(POTG)  
#### " 🙄 나, NLP에 재능이 있어버릴지도..?" 
CV와는 다른 접근방식에 흥미를 느꼈습니다! 그런데 결국 가장 잘 나올 것 같은 말을 통계적으로 찾아 붙이는 거라면 독특하고 창의적인 가사는 나오기 힘든 것이 아닐까요?

### 3-2. 어려웠던 점과 극복방안  

#### 3-2-1. NLP 모델 구조의 심도없는 이해

우리가 오늘 만든 모델은 임베딩 레이어 한 개, LSTM 레이어 두 개,  
Danse 레이어 한 개로 이루어져 있습니다.

<img src="./img/rnn2.PNG"></img>

하지만 저는 아직 RNN 이 뭔지, 왜 DANSE 모델을 LInear 라 하는지,  
모르는 것 투성입니다.

모델을 이해하지 못한 채 학습을 시키려니, 수 많은 의구심이 들었습니다.

hidden size가 왜 필요하지..? 등 

우선은 NLP 자체에 대한 맥락을 이해하고, 하나씩 배워나가려 합니다.

#### 3-2-2. val-loss 부족 

validation - loss 를 2.2 이하로 떨어뜨리는 것이 어려웠습니다.  
아직 val-loss가 무엇을 통해 줄어드는 것인지 직관적으로 이해하지 못했습니다.  
저는 다양한 시도를 해봤지만, 결정적인 영향을 끼친 것은 아래와 같습니다.

- embedding size
- hidden size 

이 두 속성의 값을 각각 512 와 2048로 키웠더니 val-loss 의 값이 유의미하게 내려갔습니다.

### 3-3. 추후  

평소 문학이나 글을 좋아하는지라 인공지능 언어 모델이 흥미로웠습니다. 견문을 넓혀 한국어 NLP 를 시도해보고 싶습니다.
